# Example 1: PDF 테이블 정보에 대한 Recursive Retrieval 전략
- 다수의 CSV 테이블 대상으로 검색 chunk와 답변 생성 chunk 분리

In [1]:
# 윈도우에서 사용시 Ghostscript 추가 설치 필요
# https://ghostscript.com/releases/gsdnld.html
import camelot

from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import PandasQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.llms.openai import OpenAI

from llama_index.readers.file import PyMuPDFReader
from typing import List

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

# 추후 사용할 llm, 임베딩 모델 클래스 정의
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [4]:
# PDF link
# https://en.wikipedia.org/wiki/The_World%27s_Billionaires

# 파싱할 파일 경로 설정
file_path = "../data/The_World's_Billionaires.pdf"

In [5]:
# PDF파서 정의
reader = PyMuPDFReader()

In [6]:
# 업로드된 경로에서 로딩스테이지 진행한 후 다큐먼트 단위로 저장
docs = reader.load(file_path)

In [7]:
# # 다큐먼트 정보 확인
# # 읽기 부적합하게 파싱된 것 확인
# docs

In [8]:
from llama_index.core import Settings
#노드변환 및 파싱
doc_nodes = Settings.node_parser.get_nodes_from_documents(docs)

In [9]:
# text만 query하는 engine 정의
vector_index0 = VectorStoreIndex(doc_nodes)
vector_query_engine0 = vector_index0.as_query_engine()

In [10]:
response = vector_query_engine0.query(
    "How many billionaires were there in 2009?"
)

In [11]:
# # 답변 생성시 사용된 node 확인
# print(response.source_nodes[0].node.get_content())

In [12]:
print(str(response))

A total of 793 billionaires were listed in 2009.


In [13]:
# naive-RAG의 잘못된 retrieve 예시
response = vector_query_engine0.query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

$195 billion


In [14]:
# # 답변 생성시 사용된 node 확인
# print(response.source_nodes[0].node.get_content())

- 기본적인 PDF파싱모듈로는 테이블 등 Text-Only 가 아닌 문서에 대한 정보 해석력이 떨어지는 것을 확인
- Table정보를 따로 추출하여 답하는 방식 필요

In [15]:
# pdf의 테이블파싱
def get_tables(path: str, pages: List[int]):
    table_dfs = []
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        for table in table_list:
            table_df = table.df
            table_df = (
                table_df.rename(columns=table_df.iloc[0])
                .drop(table_df.index[0])
                .reset_index(drop=True)
            )
            table_dfs.append(table_df)
    return table_dfs

In [16]:
table_dfs = get_tables(file_path, pages=[3,4,25])

In [17]:
# 3, 4, 24페이지에서 파싱된 테이블 개수확인
len(table_dfs)

4

In [18]:
# #파싱 결과 확인
# table_dfs[0]

In [19]:
# #파싱 결과 확인
# table_dfs[1]

In [20]:
# #파싱 결과 확인
# table_dfs[-1]

### 각 테이블별로 답해주는 담당 라마인덱스 쿼리엔진 구현

테이블이 수천 수만개일 때, 모든 유저 쿼리에 대해 수만개의 테이블을 매번 조회하는 것은 실용성 없는 Naive한 접근방식(자원은 무한하지 않다).

그렇기 때문에,
1. 사용자의 질문과 관련된 테이블을 먼저 찾고
2. 찾은 테이블을 기준으로 사용자의 질문에 답할 수 있는 정보를 발췌하여 답변 생성.

In [21]:
llm = OpenAI(model="gpt-3.5-turbo")

# pandas df 전용 query 엔진
df_query_engines = [
    PandasQueryEngine(table_df, llm=llm) for table_df in table_dfs
]

In [22]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[0].query(
    "What's the net worth of the second richest billionaire in 2024?"
)
print(str(response))

$195 billion


In [23]:
# 상응하는 테이블 지정해서 답변 요구
response = df_query_engines[1].query(
    "What's the net worth of the second richest billionaire in 2023?"
)
print(str(response))

$180 billion


In [24]:
response = df_query_engines[3].query(
    "How many billionaires were there in 2009?"
)
print(str(response))

793


질문별로 담당하는 쿼리엔진을 부여하는 것으로 heuristic하게 서칭 스페이스를 줄이고 시작할 수 있는 것 확인  
but, 질문별 담당 쿼리엔진 선택 자동화 필요

In [25]:
# 쿼리엔진별 요약문 생성
summaries = [
    (
        "This node provides information about the world's richest billionaires"
        " in 2024"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2023"
    ),
    (
        "This node provides information about the world's richest billionaires"
        " in 2022"
    ),
    # (
    #     "This node provides information about the world's richest billionaires"
    #     " in 2020"
    # ),
    (
        "This node provides information on the number of billionaires and"
        " their combined net worth from 2000 to 2023."
    ),
]

#생성된 요약문 별 노드단위 생성
df_nodes = [
    IndexNode(text=summary, index_id=f"pandas{idx}")
    for idx, summary in enumerate(summaries)
]

#요약노드 <-> 쿼리엔진 매핑
df_id_query_engine_mapping = {
    f"pandas{idx}": df_query_engine
    for idx, df_query_engine in enumerate(df_query_engines)
}

In [26]:
#생성된 노드 확인
df_nodes[0]

IndexNode(id_='782cbc03-7015-43c7-bfc2-6b1c6e156e33', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="This node provides information about the world's richest billionaires in 2024", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', index_id='pandas0', obj=None)

In [27]:
# 상위레벨 벡터스토어인덱스 정의
# 최상위 task는 유저의 질문에 잘 답할 수 있는 node를 찾는 것이 됨
vector_index = VectorStoreIndex(df_nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1) # 답을 가장 잘 할 수 있는 노드를 찾아야해서 top_K가 1이 됨

In [28]:
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=df_id_query_engine_mapping,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")

# 최종 query 엔진
query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever, response_synthesizer=response_synthesizer
)

In [29]:
response = query_engine.query(
    "What's the net worth of the second richest billionaire in 2023?"
)

Retrieving with query id None: What's the net worth of the second richest billionaire in 2023?
Retrieved node with id, entering: pandas1
Retrieving with query id pandas1: What's the net worth of the second richest billionaire in 2023?
Got response: $180 billion


In [30]:
response.source_nodes[0].node.get_content()

"Query: What's the net worth of the second richest billionaire in 2023?\nResponse: $180 billion"

In [31]:
str(response)

'$180 billion'

In [32]:
response = query_engine.query("How many billionaires were there in 2009?")

Retrieving with query id None: How many billionaires were there in 2009?
Retrieved node with id, entering: pandas3
Retrieving with query id pandas3: How many billionaires were there in 2009?
Got response: 15    793
Name: Number of billionaires, dtype: object


In [33]:
str(response)

'There were 793 billionaires in 2009.'

- 서머리 텍스트로 recursive retriever 모듈로 하여금 우리가 찾고자 하는 문서를 자동으로 판별해서 해당 쿼리엔진을 기반으로만 답하게 하는 것이 가능한 것 확인
  
- Searching 공간을 최적화하는 것이 RAG 성능에 중요한 영향

### Pinecone DB에 있는 데이터와 연계

In [36]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import re
import os

pinecone_api_key = os.environ.get('PINECONE_API_KEY')

In [35]:
from datasets import load_dataset
# 데이터 로드
dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:1000]')
data = dataset.to_pandas()[['id', 'text', 'title']].drop_duplicates(subset='text', keep='first')

In [37]:
def clean_up_text(content: str) -> str:
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    content = re.sub(r'\\n|  —|——————————|—————————|—————|\\u[\dA-Fa-f]{4}|\uf075|\uf0b7', "", content)

    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

In [38]:
from llama_index.core import Document, VectorStoreIndex

documents = [Document(
    text=clean_up_text(row['text']),
    doc_id=row['id'],
    extra_info={'title': row['title']}
) for _, row in data.iterrows()]

In [39]:
data.head()

,id,text,title
0,5,"제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대...",지미 카터
1,9,"수학(數學, , 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루...",수학
2,10,"수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는...",수학 상수
3,19,"문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과...",문학
4,20,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,나라 목록


In [41]:
# 파인콘 인덱스 생성
from time import sleep

pc = Pinecone(api_key=pinecone_api_key)

index_name="quickstart"

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(
   name=index_name,
   dimension=1536,
   metric="dotproduct",
   spec=ServerlessSpec(
       cloud='aws',
       region='us-east-1'
   )
)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)
index = pc.Index(index_name)
sleep(1)
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [42]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

vector_store = PineconeVectorStore(pinecone_index=index)
# 벡터스토어 인덱스에 들어가는 스토리지 컴포넌트 정의
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,
    service_context=service_context
)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15204\3549792048.py:11: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/2048 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/1474 [00:00<?, ?it/s]

In [43]:
# Query 엔진 가변화 with OpenAIAgent(function call)
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
agents = {}

for title in `.title:
    vector_query_engine = index.as_query_engine(vector_store_kwargs={"filter": {"title": title}})
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"{title}에 대해서 물어볼 때 사용"
                ),
            ),
        ),
    ]

    function_llm = OpenAI(model="gpt-4-turbo-preview", temperature=0)
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
    )

    agents[title] = agent

In [45]:
# #생성된 에이전트 확인
# agents

In [47]:
# 에이전트 선택을 위한 에이전트 summary
nodes = []
for title in data.title:
    doc_summary = (
        f"이것은 {title}과 관련된 내용이 있습니다. "
        f"{title}과 관련된 내용을 확인하는 용도로 이 인덱스를 사용하세요."
    )
    node = IndexNode(text=doc_summary, index_id=title)
    nodes.append(node)

In [48]:
# 에이전트 선택 인덱스(노드) 정의
vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)

# 에이전트 자체를 쿼리엔진으로 하여 선택된 에이전트가 쿼리 엔진 역할을 하도록 구성
recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    query_engine_dict=agents,
    verbose=True,
)

response_synthesizer = get_response_synthesizer(response_mode="compact")


query_engine = RetrieverQueryEngine.from_args(
    recursive_retriever,
    response_synthesizer=response_synthesizer
)

In [49]:
# 해당 문서에서만 답변 가능한 굉장히 구체적인 질문 테스트
response = query_engine.query("셀빅에 대해 알려줘")

Retrieving with query id None: 셀빅에 대해 알려줘
Retrieved node with id, entering: 셀빅
Retrieving with query id 셀빅: 셀빅에 대해 알려줘
Added user message to memory: 셀빅에 대해 알려줘
=== Calling Function ===
Calling function: vector_tool with args: {"input":"셀빅"}
Got output: CellVic was a type of PDA produced in South Korea by jTel, later acquired by Kolon and renamed Cellvic. It was significant as the first PDA operating system developed in Korea, tailored to the local environment. jTel focused on securing applications through regular competitions and support for individual developers, resulting in a variety of applications. The device lineup ranged from the lightweight CellVic i to the advanced smartphone mycube. However, after Kolon took over, the company ceased further support due to competition in the smartphone market with PocketPC devices, leading to discontinuation in 2004.

Got response: 셀빅(CellVic)은 대한민국에서 jTel에 의해 생산된 PDA 유형이었으며, 나중에 Kolon에 인수되어 Cellvic으로 이름이 바뀌었습니다. 셀빅은 한국에서 개발된 첫 PDA 운영 체제로, 현지 

- Decoupling 전략은 모든 RAG에 필수적